In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Defining the path of the file
path = 'C:/Users/Humberto/Google Drive/Clube da aposta/Analise de dados/Gestor de banca/'

In [4]:
def betfair_results(spreadsheet):
    dataframe = pd.read_excel(spreadsheet)

    #Tyding the spreadsheet
    dataframe.rename(columns={'Persistência    ': 'Persistência', 'Probabilidades req. ': 'Probabilidades req.' }, inplace=True)
    dataframe.dropna(how='all', axis=1, inplace=True)
    dataframe['Aposta colocada'] = pd.to_datetime(dataframe['Aposta colocada'])
    dataframe.set_index('ID da Aposta', inplace=True)
    dataframe.sort_index(inplace=True)
    #dataframe.sort_values('ID da Aposta', axis=0, inplace=True)

    #Setting the formats for floats
    dataframe['Probabilidades req.'] = dataframe['Probabilidades req.'].apply(lambda x: float(x.replace(',','.')))
    dataframe['Média de probabilidades correspondidas'] = dataframe['Média de probabilidades correspondidas'].apply(lambda x: float(x.replace(',','.')))
    dataframe['Lucro/Prejuízo(BRL)'] = dataframe['Lucro/Prejuízo(BRL)'].apply(lambda x: float(x.replace(',','.').replace(')','').replace('(','-')))
    dataframe['Parada(BRL)'] = dataframe['Parada(BRL)'].apply(lambda x: float(x.replace(',','.')))

    #Creating some features

    def get_market(text):
        '''Just a string function to return the market'''

        market = text.split(' / ')[-1]

        return market

    def get_match(text):
        '''Just a string function to return the match'''

        match = text.split(' / ')[0] + ' ' + text.split(' / ')[1]

        return match

    dataframe['Tipo de Mercado'] = dataframe['Mercado'].apply(get_market)
    dataframe['Partida'] = dataframe['Mercado'].apply(get_match)

    #Start looping
    matches_list = dataframe['Partida'].unique()
    for match in matches_list:
        match_df = dataframe[dataframe['Partida'] == match].copy()

        #Now filtering by market
        markets = match_df['Tipo de Mercado'].unique()
        for market in markets:

            if market == 'Probabilidades':
                print('Creating Betfair results for {}-{}'.format(match, market), end='\r')

                #Infos
                market_df = match_df[match_df['Tipo de Mercado'] == market]
                home_team = market_df.loc[market_df.index[0],'Mercado'].split(' / ')[1].split(' x ')[0]
                away_team = market_df.loc[market_df.index[0],'Mercado'].split(' / ')[1].split(' x ')[1]
                home_pl = float(0)
                draw_pl = float(0)
                away_pl = float(0)

                #adding columns
                market_df[home_team] = home_pl
                market_df['Draw'] = draw_pl
                market_df[away_team] = away_pl

                for key, row in market_df.iterrows():

                    selection = row['Selecção']
                    stake = row['Parada(BRL)']
                    odds = row['Média de probabilidades correspondidas']
                    back_or_lay = row['Tipo de licitação']

                    #If the bet was corresponded
                    if row['Persistência'] == 'C':

                        #If it's a back
                        if back_or_lay == 'Apostar a favor':

                            #Now the selections
                            if  selection == home_team:
                                home_pl += stake * (odds - 1)
                                draw_pl -= stake
                                away_pl -= stake

                            if  selection == 'Empate':
                                home_pl -= stake
                                draw_pl += stake * (odds - 1)
                                away_pl -= stake

                            if  selection == away_team:
                                home_pl -= stake
                                draw_pl -= stake
                                away_pl += stake * (odds - 1)

                        #If it's a lay
                        if back_or_lay == 'Apostar contra':

                            #Now the selections
                            if  selection == home_team:
                                home_pl -= stake * (odds - 1)
                                draw_pl += stake
                                away_pl += stake

                            if  selection == 'Empate':
                                home_pl += stake
                                draw_pl -= stake * (odds - 1)
                                away_pl += stake

                            if  selection == away_team:
                                home_pl += stake
                                draw_pl += stake
                                away_pl -= stake * (odds - 1)

                        #Adding the profit and loss
                        market_df.loc[key, home_team] = home_pl
                        market_df.loc[key, "Draw"] = draw_pl
                        market_df.loc[key, away_team] = away_pl

                        market_df = market_df.round(2)
                        

                market_df = market_df.reset_index().set_index('Aposta colocada', drop=True)
                plt.figure(figsize=(10,5))
                plt.plot(market_df[home_team], color='Blue', label=home_team)
                plt.plot(market_df['Draw'], color='gray', label='Draw')
                plt.plot(market_df[away_team], color='Red', label=away_team)
                plt.axhline(0, color='lightgray', linestyle='dashed')
                
                plt.xticks(rotation=45)
                plt.legend()

                #Exporting Excel and Figure
                market_df.to_excel(path + '{} - {}.xlsx'.format(match, market), encoding='utf-8')
                plt.savefig(path + '{} - {}.png'.format(match, market))

In [ ]:
betfair_results(path + 'BetHistory.xls')

C:\Users\Humberto\Anaconda3\lib\site-packages\ipykernel\__main__.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Humberto\Anaconda3\lib\site-packages\ipykernel\__main__.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Humberto\Anaconda3\lib\site-packages\ipykernel\__main__.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

In [22]:
df.columns

Index(['Tipo de Mercado', 'Mercado', 'Selecção', 'Tipo de licitação',
       'Aposta colocada', 'Unnamed: 6', 'Persistência    ',
       'Probabilidades req. ', 'Parada(BRL)', 'Responsabilidade(BRL)',
       'Média de probabilidades correspondidas', 'Lucro/Prejuízo(BRL)'],
      dtype='object')